本周只有一个代码实践题

### <评阅点>     
1. 完成代码并无bug  50‘   
2. 代码有部分bug   40'   
3. 代码有重大bug   30‘
4. 代码不完整  20'   
5. 其余 0‘   

以下代码课用来加载 预先训练好的模型,你需要只需要修改模型的存放路径即可（第二行代码）

In [41]:
import torch, random
import numpy as np
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
from torch.utils.data import DataLoader,Dataset

第一步 使用以下链接下载相应预训练模型

https://convaisharables.blob.core.windows.net/lsp/multiref/small_ft.pkl

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
weights = torch.load('../Lecture13/small_ft.pkl')
medium_config = GPT2Config(n_embd = 768,n_layer = 12, n_head = 12)
model = GPT2LMHeadModel(medium_config)

weights['lm_head.weight'] = weights['lm_head.decoder.weight']
weights.pop('lm_head.decoder.weight',None)

model.load_state_dict(weights)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

你需要写一个推理函数，这个函数接收一个英文句子为输入，输出一个回应。

### 测试语句 
一轮对话   
1. Does money buy happiness ?   
2. What is the best way to buy happiness?   

一轮对话   
1. what is the meaning of a godd life ?   
2. How to be a good person ?   

这个我应该用p(w1|w2)来计算每句话的概率, 但是没语料库

计算概率用的是 (p(w1) + p(w2) + ... + p(wn)) / n

In [76]:
def token_predict(sentence, k_sampling):
    
    input_ids = sum([tokenizer.encode(sentence)],[])
    
    input_ids = pad_sequence([torch.tensor(input_ids,dtype=torch.long)],batch_first=True,padding_value=0)
    
    logits, _ = model(input_ids=input_ids)
    lm_logits= logits[0][-1]
    softmax = F.softmax(lm_logits, dim=-1)
    softmax, indices = torch.sort(softmax, descending = True)
    return [tokenizer.decode([indices[i]]) for i in range(k_sampling)], [float(softmax[i]) for i in range(k_sampling)]
    
def predict(sentence, k_sampling=2):
    end_text = '<|endoftext|>'
    
    heap = [(sentence, 1, 1)]
    end_of_text = []
    
    while heap:
        sentence, sentence_probility, num = heap.pop(-1)
        
        next_token_list, probility_list = token_predict(sentence, k_sampling)
        if next_token_list[0] == end_text:
            end_of_text.append((sentence + end_text, sentence_probility + probility_list[0], num + 1))
            continue
        else:
            heap.extend([(sentence + token, sentence_probility + p, num + 1) for token, p in zip(next_token_list, probility_list) if token != end_text])
            end_of_text.extend([(sentence + token, sentence_probility + p, num + 1) for token, p in zip(next_token_list, probility_list) if token == end_text])
            if len(end_of_text) + len(heap) > k_sampling:
                probility_threshold = sorted([x[1] / x[2] for x in heap + end_of_text], reverse=True)[:k_sampling][-1]
                heap = list(filter(lambda x: x[1] / x[2] >= probility_threshold, heap))
                end_of_text = list(filter(lambda x: x[1] / x[2] >= probility_threshold, end_of_text))
                
    return [(x[0], x[1] / x[2]) for x in sorted(end_of_text, key=lambda x: x[1]/ x[2], reverse=True)]

In [79]:
predict('Does money buy happiness ?<|endoftext|>')

[('Does money buy happiness ?<|endoftext|>Money buys happiness?<|endoftext|>',
  0.5304691592852274),
 ('Does money buy happiness ?<|endoftext|>Money buys happiness.c<|endoftext|>',
  0.36382991448044777)]

In [80]:
predict('Does money buy happiness ? Money buys happiness? What is the best way to buy happiness?<|endoftext|>')

[('Does money buy happiness ? Money buys happiness? What is the best way to buy happiness?<|endoftext|>Money buys happiness.<|endoftext|>',
  0.44108057270447415),
 ('Does money buy happiness ? Money buys happiness? What is the best way to buy happiness?<|endoftext|>Money buys happiness, money DOES not buy you.lol<|endoftext|>',
  0.34112982566540057)]

In [93]:
predict('what is the meaning of a good life ?<|endoftext|>')

[('what is the meaning of a good life ?<|endoftext|>Drugs, Money,Drugs,Money<|endoftext|>',
  0.4793877430598844),
 ('what is the meaning of a good life ?<|endoftext|>Drugs, Money,Drugs,Drug intercourse, Money,life<|endoftext|>',
  0.4553552003344521)]

In [94]:
predict('what is the meaning of a good life ? Drugs, Money,Drugs,Money How to be a good person ?<|endoftext|>')

[("what is the meaning of a good life ? Drugs, Money,Drugs,Money How to be a good person ?<|endoftext|>So what's the meaning of life?<|endoftext|>",
  0.543706813827157),
 ("what is the meaning of a good life ? Drugs, Money,Drugs,Money How to be a good person ?<|endoftext|>I'm a Drug dealer.<|endoftext|>",
  0.28902812860906124)]